# Town Hall Meetings by Appalachian Representatives

In [117]:
import json
import os

DATA_DIR = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), 'data')
MEETINGS_JSON_PATH = os.path.join(DATA_DIR, 'official_meetings.json')
AHCA_VOTES_MAY_JSON_PATH = os.path.join(DATA_DIR, 'ahca-votes-256.json')
AHCA_VOTES_MARCH_JSON_PATH = os.path.join(DATA_DIR, 'ahca-votes-192.json')

with open(MEETINGS_JSON_PATH) as f:
    meetings_json = json.load(f)

def flatten(raw_data):
    meetings = []
    officials = []
    
    if 'objects' in raw_data:
        officials_raw = raw_data['objects']
    else:
        officials_raw = raw_data
        
    for official in officials_raw:
        officials.append({
            'name': official['name'],
            'party': official['party'],
            'ocd_id': official['office']['division']['ocd_id'],  
            'num_meetings': len(official['meetings'])
        })
            
        for meeting in official['meetings']:

            meetings.append({
                'name': official['name'],
                'party': official['party'],
                'ocd_id': official['office']['division']['ocd_id'],
                'date': meeting['date'],
                'meeting_type': meeting['meeting_type'],
                'meeting_id': meeting['id'],
            })

    return officials, meetings

# The raw JSON contains meetings nested under officials.  Flatten this for easier
# analysis.
officials_raw, meetings_raw = flatten(meetings_json)

In [118]:
import pandas as pd

# Read meetings into a dataframe
meetings = pd.DataFrame.from_dict(meetings_raw)

# Read votes into a dataframe

with open(AHCA_VOTES_MAY_JSON_PATH) as f:
    ahca_votes_may_raw = json.load(f)['votes']

ahca_votes_may = pd.DataFrame.from_dict(ahca_votes_may_raw)

with open(AHCA_VOTES_MARCH_JSON_PATH) as f:
    ahca_votes_march_raw = json.load(f)['votes']
    
ahca_votes_march = pd.DataFrame.from_dict(ahca_votes_march_raw)

In [119]:
import numpy as np

officials = pd.DataFrame.from_dict(officials_raw)

officials_with_votes = officials\
    .merge(ahca_votes_march, how='left', on='ocd_id')\
    .rename(columns={'vote_position': 'ahca_vote_march'})\
    .drop('id', axis=1)\
    .merge(ahca_votes_may, how='left', on='ocd_id')\
    .rename(columns={'vote_position': 'ahca_vote_may'})\
    .drop('id', axis=1)\


officials_with_votes

,name,num_meetings,ocd_id,party,ahca_vote_march,ahca_vote_may
0,John K. Delaney,1,ocd-division/country:us/state:md/cd:6,Democratic,No,No
1,H. Morgan Griffith,0,ocd-division/country:us/state:va/cd:9,Republican,Yes,Yes
2,Harold Rogers,0,ocd-division/country:us/state:ky/cd:5,Republican,Yes,Yes
3,Bob Goodlatte,0,ocd-division/country:us/state:va/cd:6,Republican,Yes,Yes
4,David P. Roe,4,ocd-division/country:us/state:tn/cd:1,Republican,Yes,Yes
5,Diane Black,4,ocd-division/country:us/state:tn/cd:6,Republican,Yes,Yes
6,David B. McKinley,1,ocd-division/country:us/state:wv/cd:1,Republican,Yes,Yes
7,Robert B. Aderholt,0,ocd-division/country:us/state:al/cd:4,Republican,Yes,Yes
8,Robert Pittenger,2,ocd-division/country:us/state:nc/cd:9,Republican,Yes,Yes
9,Trey Gowdy,1,ocd-division/country:us/state:sc/cd:4,Republican,Yes,Yes


## Representatives who broke with their party

In [120]:
# Who were the Republicans who broke with their party?
officials_with_votes[(officials_with_votes['party'] == "Republican") & (officials_with_votes['ahca_vote_may'] == "No")]

,name,num_meetings,ocd_id,party,ahca_vote_march,ahca_vote_may
22,Thomas Massie,0,ocd-division/country:us/state:ky/cd:4,Republican,No,No
48,Barbara Comstock,2,ocd-division/country:us/state:va/cd:10,Republican,Yes,No
51,John Katko,0,ocd-division/country:us/state:ny/cd:24,Republican,Yes,No
52,David P. Joyce,0,ocd-division/country:us/state:oh/cd:14,Republican,Yes,No


In [121]:
# Who were the Democrats who broke with their party?
officials_with_votes[(officials_with_votes['party'] == "Democratic") & (officials_with_votes['ahca_vote_may'] == "Yes")]

,name,num_meetings,ocd_id,party,ahca_vote_march,ahca_vote_may


## Representatives who switched from the March vote to the May vote

In [122]:
officials_with_votes[(officials_with_votes['ahca_vote_march'] != officials_with_votes['ahca_vote_may'])]

,name,num_meetings,ocd_id,party,ahca_vote_march,ahca_vote_may
31,Mo Brooks,1,ocd-division/country:us/state:al/cd:5,Republican,No,Yes
48,Barbara Comstock,2,ocd-division/country:us/state:va/cd:10,Republican,Yes,No
51,John Katko,0,ocd-division/country:us/state:ny/cd:24,Republican,Yes,No
52,David P. Joyce,0,ocd-division/country:us/state:oh/cd:14,Republican,Yes,No


## Representatives who had no meetings

In [129]:
officials_no_meetings = officials[officials['num_meetings'] == 0]\
    .sort_values(by='party')
officials_no_meetings

,name,num_meetings,ocd_id,party
60,Bennie G. Thompson,0,ocd-division/country:us/state:ms/cd:2,Democratic
63,David Scott,0,ocd-division/country:us/state:ga/cd:13,Democratic
1,H. Morgan Griffith,0,ocd-division/country:us/state:va/cd:9,Republican
59,"Charles J. ""Chuck"" Fleischmann",0,ocd-division/country:us/state:tn/cd:3,Republican
56,Patrick J. Tiberi,0,ocd-division/country:us/state:oh/cd:12,Republican
52,David P. Joyce,0,ocd-division/country:us/state:oh/cd:14,Republican
51,John Katko,0,ocd-division/country:us/state:ny/cd:24,Republican
50,Tom Marino,0,ocd-division/country:us/state:pa/cd:10,Republican
47,Glenn Thompson,0,ocd-division/country:us/state:pa/cd:5,Republican
45,Scott DesJarlais,0,ocd-division/country:us/state:tn/cd:4,Republican
